# Convolution


In [1]:
import torch
import torch.nn.functional as F

input = torch.tensor([[1,2,3,4,5],
                      [2,3,4,5,6],
                      [3,4,5,6,7],
                      [4,5,6,7,8],
                      [5,6,7,8,9]])

kernel = torch.tensor([[1,0,1], # 卷积核，大小为 3x3
                       [0,1,0],
                       [1,0,1]])

input = torch.reshape(input, (1, 1, 5, 5)) # 将输入数据调整为 4D 张量，形状为 (batch_size, channels, height, width)
kernel = torch.reshape(kernel, (1, 1, 3, 3)) # 将卷积核调整为 4D 张量，形状为 (out_channels, in_channels, height, width)

output = F.conv2d(input, kernel, stride=1, padding=0) # 使用 conv2d 函数进行卷积操作，步长为 1，填充为 0
print(output) # 输出卷积结果

tensor([[[[15, 20, 25],
          [20, 25, 30],
          [25, 30, 35]]]])


In [3]:
import torch
from torch import nn
from torch.nn import Conv2d
from torch.utils.data import DataLoader
import torchvision
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10(root="./data", train=False, transform=torchvision.transforms.ToTensor(), download=False)
dataloader = DataLoader(dataset, batch_size=64)

class Mynet(nn.Module):
    def __init__(self):
        super(Mynet, self).__init__()
        self.conv1 = Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1, padding=0) # 定义第一层卷积，输入通道数为 3，输出通道数为 6，卷积核大小为 3x3，步长为 1，填充为 0

    def forward(self, x):
        x = self.conv1(x) # 将输入数据通过第一层卷积进行处理
        return x    

writer = SummaryWriter("logs") # 创建 SummaryWriter 实例，用于记录训练过程中的数据

mynet = Mynet() # 创建 Mynet 实例
global_step = 0 # 初始化全局步数
for data in dataloader: # 遍历数据加载器中的数据
    imgs, targets = data # 获取输入图像和对应的标签
    output = mynet(imgs) # 将输入图像通过 Mynet 进行前向传播，得到输出结果
    writer.add_images("input", imgs, global_step) # 将输入图像添加到 TensorBoard 中，标签为 "input"，步数为 0
    output = output.reshape(-1, 3, 30, 30) # 将输出结果调整为适合 TensorBoard 显示的形状，假设输出通道数为 6，图像大小为 30x30
    writer.add_images("output", output, global_step) # 将输出结果添加到 TensorBoard 中，标签为 "output"，步数为 0
    global_step += 1 # 增加全局步数

